<a href="https://colab.research.google.com/github/sheffieldcl/NIHL-Training/blob/main/week6b_sz_nlp_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook demonstrates how to use a vector database and large language models (LLM) to perform semantic similarity comparisions and whole text searches.

1 Install the Python packages


*   biopython - Python API to access PubMed database and GenBank database by NCBI, NIH
*   qdrant-client - Python client API for Qdrant vector search engine for whole text search
*   openai - the OPEN AI client API for ChatGPT
*   sentence-transformers - it provides an easy method to compute dense vector representations for sentences, paragraphs, and images



In [ ]:
! pip install biopython
! pip install qdrant-client
! pip install openai
! pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 re

# 2 Fetch the COVID-19 research paper abstracts from PubMed

In [ ]:
from Bio import Entrez
import pandas as pd
import numpy as np

The function to search PubMed using Python API.

Please change the Entrez.email to your email address, because PubMed uses the email address for data control, if all people use the identical email, the search will be suspended soon.

In [ ]:
def search(query, retmax='1000'):
    # please use your own email address
    Entrez.email = 'my_email@gmail.com'
    handle = Entrez.esearch(db='pubmed', sort='relevance', retmax=retmax, retmode='xml', term=query)
    results = Entrez.read(handle)
    return results


def fetch_details(id_list):
    # convert the list of IDs into a long string, each ID separated by ','
    ids = ','.join(id_list)
    Entrez.email = 'my_email@gmail.com'
    handle = Entrez.efetch(db='pubmed',retmode='xml',id=ids)
    results = Entrez.read(handle)
    # the return is a complex json structure data structure
    return results

In [ ]:
results = search('COVID-19 vaccine safety', '2000') # you can change the max retrieval number, max retrieval # is 10,000
print(results.keys())
print(f"Total hits: {results['Count']}, retrieved # of paper ID: {results['RetMax']}")

dict_keys(['Count', 'RetMax', 'RetStart', 'IdList', 'TranslationSet', 'QueryTranslation'])
Total hits: 7659, retrieved # of paper ID: 2000


After we find the relevant papers, we fetch the data and store them into a pandas dataframe

In [ ]:
# get the retrieved paper IDs
study_ids = results['IdList']

# initiate the list to hold the data in different fields
title_list= []
abstract_list=[]
journal_list = []
language_list =[]
pubdate_year_list = []
pubdate_month_list = []

# fetch the paper details, the returned is a Bio.Entrez.Parser.DictionaryElement object
studies = fetch_details(study_ids)

Next, we need to chunk the data, and fetch the detailed records chunk by chunk from PubMed.  

In [ ]:
# define the chunk size, it means how many paper we will fetch from PubMed at a single time
chunk_size = 100

# We used the search function to get the PubMed IDs of the relevant papers
# Next, we will use the fetch_details function to fetch the paper details by chunks
# This method significantly lowers the Internet traffic

for chunk_i in range(0, len(study_ids), chunk_size):
    chunk = study_ids[chunk_i:chunk_i + chunk_size]
    papers = fetch_details(chunk)
    for i, paper in enumerate (papers['PubmedArticle']):
        title_list.append(paper['MedlineCitation']['Article']['ArticleTitle'])
        try:
            abstract_list.append(paper['MedlineCitation']['Article']['Abstract']['AbstractText'][0])
        except:
            abstract_list.append('No Abstract')
        journal_list.append(paper['MedlineCitation']['Article']['Journal']['Title'])
        language_list.append(paper['MedlineCitation']['Article']['Language'][0])
        try:
            pubdate_year_list.append(paper['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']['Year'])
        except:
            pubdate_year_list.append('No Data')
        try:
            pubdate_month_list.append(paper['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']['Month'])
        except:
            pubdate_month_list.append('No Data')

In [ ]:
# show the paper titles of the first ten hits
title_list[0:10]

['Safety and Efficacy of the BNT162b2 mRNA Covid-19 Vaccine.',
 'COVID-19 vaccine development: milestones, lessons and prospects.',
 'Batch-dependent safety of the BNT162b2 mRNA COVID-19 vaccine.',
 '[Comment] COVID‑19 vaccine safety.',
 'COVID-19 vaccine hesitancy: misinformation and perceptions of vaccine safety.',
 'COVID-19 vaccine safety and efficacy in patients with immune-mediated inflammatory disease: Review of available evidence.',
 'Safety and Efficacy of a Third Dose of BNT162b2 Covid-19 Vaccine.',
 'COVID-19 vaccine and pregnancy: A safety weapon against pandemic.',
 'Efficacy and Safety of the RBD-Dimer-Based Covid-19 Vaccine ZF2001 in Adults.',
 'Safety, Immunogenicity, and Efficacy of the BNT162b2 Covid-19 Vaccine in Adolescents.']

Next step, we will load the data from list to a pandas dataframe.

Pandas DataFrame provides a method to zip the column lists and attach the column names to them

In [ ]:
# load the fetched data into a pandas dataframe

df = pd.DataFrame(list(zip(title_list, abstract_list, journal_list, language_list, pubdate_year_list, pubdate_month_list)),
                  columns=['Title', 'Abstract', 'Journal', 'Language', 'Year','Month'],)

# Next, we do some data cleaning

df['Month'].replace('Jan', '01', inplace=True)
df['Month'].replace('Feb', '02', inplace=True)
df['Month'].replace('Mar', '03', inplace=True)
df['Month'].replace('Apr', '04', inplace=True)
df['Month'].replace('May', '05', inplace=True)
df['Month'].replace('Jun', '06', inplace=True)
df['Month'].replace('Jul', '07', inplace=True)
df['Month'].replace('Aug', '08', inplace=True)
df['Month'].replace('Sep', '09', inplace=True)
df['Month'].replace('Oct', '10', inplace=True)
df['Month'].replace('Nov', '11', inplace=True)
df['Month'].replace('Dec', '12', inplace=True)

# use "np.nan" to replace "No Data"
df['Month'].replace('No Data', np.nan, inplace=True)

In [ ]:
# total rows in the dataframe
len(df)

2000

In [ ]:
# show a few rows
df.head()

,Title,Abstract,Journal,Language,Year,Month
0,Safety and Efficacy of the BNT162b2 mRNA Covid...,Severe acute respiratory syndrome coronavirus ...,The New England journal of medicine,eng,2020,12
1,"COVID-19 vaccine development: milestones, less...",With the constantly mutating of SARS-CoV-2 and...,Signal transduction and targeted therapy,eng,2022,05
2,Batch-dependent safety of the BNT162b2 mRNA CO...,No Abstract,European journal of clinical investigation,eng,2023,08
3,[Comment] COVID‑19 vaccine safety.,"In response to the SARS‑CoV‑2 outbreak, and th...",International journal of molecular medicine,eng,2020,11
4,COVID-19 vaccine hesitancy: misinformation and...,"Despite COVID-19's devastating toll, many Amer...",Human vaccines & immunotherapeutics,eng,2022,12


# 3 Convert the data into the dictionary format

First we convert the data from the dataframe to a list of dictionaries. Each row is an element of the list, and each column is a key-value pair

In [ ]:
# convert the data to a list of dictionaries
data = df.to_dict('records') # get a list of dicts

In [ ]:
# we show the abstract of the first row, you can cross-check with the above table
print(data[0]['Title'])
print(data[0]['Abstract'])

Safety and Efficacy of the BNT162b2 mRNA Covid-19 Vaccine.
Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) infection and the resulting coronavirus disease 2019 (Covid-19) have afflicted tens of millions of people in a worldwide pandemic. Safe and effective vaccines are needed urgently.


# 4 Import the text data into a vector database for LLM search

Qdrant is an Open Source vector database with in-built vector similarity search engine. Qdrant is a fast and reliable vector database even under high load in production environment. Qdrant provides convenient API to store, search and manage vectors along with the associated payload for the vectors.

Sentence Transformers (a.k.a. SBERT) is the go-to Python module for accessing, using, and training state-of-the-art text and image embedding models. It can be used to compute embeddings using Sentence Transformer models or to calculate similarity scores using Cross-Encoder models.

In [ ]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

Use the Sentence Transformer (a.k.a. SBERT) to encode the text into embedding vectors. The API use the model with the keyword "all-MiniLM-L6-v2".

In [ ]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


First, we create a in-memory vector DB for fast vector searching.

Second, we create a collection for the text embedding data from the paper abstracts of paper data fectched from PubMed.

If you also want to use the data from the paper titles, you can concatenate the title text to the abstract as a whole piece of text for searching.

In [ ]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

Define the collection to encode the text
Here we compute the text similarity by Cosine similarity, Qdrant also provides other similarity algorithms:


*   Dot product: DOT
*   Cosine similarity: COSINE
*   Euclidean distance: EUCLID
*   Manhattan distance: MANHATTAN





In [ ]:

qdrant.recreate_collection(
    collection_name="covid_vaccine",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

<ipython-input-52-e356bdf41227>:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

Encode the text in the Abstract column to text embeddings.

We use a list to store the embedding data, which will be later uploaded to the vector database.

The process will take a few minutes because we will upload all the data to the in-memory database, which will consume a large amount of memory.

In [ ]:
data_points = []


# we use the encoded vectors of abstracts for search, but the database also have the data of other columns
for idx, doc in enumerate(data):
  data_points.append(models.PointStruct(id=idx, vector=encoder.encode(doc["Abstract"]).tolist(), payload=doc))

Each embedding vector is a list of the encodings of each word in the text,
The SBERT model uses a 384 dimensional vector to represent each word in the text

In [ ]:
len(data_points[0].vector)

384

In [ ]:
# load the word embeddings to the vector database

qdrant.upload_records(
    collection_name="covid_vaccine",
    records=data_points
)

<ipython-input-55-3c9bc59fe6c3>:3: DeprecationWarning: `upload_records` is deprecated, use `upload_points` instead
  qdrant.upload_records(


In [ ]:
# choose a topic you want to search
query = "safety of vector vaccine for covid-19"

# get the top 5 most relevant records
# all available data will has a score, so we should determine how many hit we want to retrieve
# if you have uploaded more than one collection to the vector database, make sure you choose the correct collection name for search

hits = qdrant.search(
    collection_name="covid_vaccine",
    query_vector=encoder.encode(query).tolist(),
    limit=10
)

In [ ]:
len(hits)

10

In [ ]:
for hit in hits:
  print(hit.payload['Title'], hit.payload['Year'], "----","score:", hit.score)
  print(hit.payload['Abstract'])
  print("="*20)

Efficacy and safety of COVID-19 vaccines: a systematic review. 2021 ---- score: 0.8385145952628738
To evaluate systematically the efficacy and safety of COVID-19 vaccines.
Vaccines for COVID-19: A Systematic Review of Feasibility and Effectiveness. 2022 ---- score: 0.7998312936857696
Many potential vaccines for COVID-19 are being studied and developed. Several studies have reported on the safety and efficacy of these vaccines. This systematic review aimed to report on the current evidence concerning the feasibility and effectiveness of vaccines for COVID-19.
Comparing reactogenicity of COVID-19 vaccines: a systematic review and meta-analysis. 2022 ---- score: 0.7907541140576533
A number of vaccines have now been developed against COVID-19. Differences in reactogenicity and safety profiles according to the vaccine technologies employed are becoming apparent from clinical trials.
A review of the safety and efficacy of current COVID-19 vaccines. 2022 ---- score: 0.7716608794813213
Vaccina

In [ ]:
# define a variable to hold the search results
# because the LLM takes list of dictionary as input, we format the data in this way

search_results = [hit.payload for hit in hits]

# 5 Use ChatGPT for retrieval augmented generation (RAG)

To use ChatGPT, you need to register an account on [OPENAI](https://openai.com/).

Then go to "Product --> API login --> API --> API keys" to get your API Key to use ChatGPT with Python. Click "Create new secret key", and copy the API key string to the os.environ["OPENAI_API_KEY"] below.


In [ ]:
import openai
from openai import OpenAI
import os

# put your own Open AI API Key here:
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# create an OpenAi client
client = OpenAI()

Define the RAG function to chat with ChatGPT

Open AI will give a credit ($3-$5) to new registered users, use ChatGPT 3.5 turbo is good enough for this lab.
To check the complete list of ChatGPT models, please visit:


https://platform.openai.com/docs/models

In [ ]:
# define the function to take the instructions and the prompt message, then return the response from ChatGPT

def get_rag_completion(prompt, role_instruct="", model="gpt-3.5-turbo"):
    messages = [
        {'role': "system", 'content': role_instruct},
        {'role': "user", 'content': prompt},
        {'role': "assistant", 'content': str(search_results)}
    ]

# Create the authorization header
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"

    }

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.1, # this is the degree of randomness of the model's output, 0 - 1.0
    )
    return response.choices[0].message.content

Next, we will tell ChatGPT that it is a medical expert for COVID-19, and its task is to provide brief response to the question within 150 words.

Then we start to ask question by the prompts.

Retrieval augmented generation (RAG) is the technology to retrieve detail information from a large language model (LLM) based on the assumption that the interactive prompting can utilize the capacity of LLM to retrieve in-depth information from its training data.

In [ ]:
role_instruct = "You are medical specialist for COVID-19. Your top priority is to explain how vaccines can protect people from COVID-19 infection. Your answer should be about 150 words"
prompt_1 = "Tell me about the safety of vector vaccine for COVID-19"
prompt_2 = "What kind of severe side effects might be when receiving mRNA vaccine for COVID-19"

In [ ]:
completion = get_rag_completion(prompt_1, role_instruct)
completion

NameError: name 'api_key' is not defined

In [ ]:
completion = get_rag_completion(prompt_2, role_instruct)
completion

APIConnectionError: Connection error.

# Next Step

Congratulations! This is the end of the tutorial.
The next step is to decide your own research topic and try to retrive the paper from PubMed with Python and get the most relevant information for your research.